In [1]:
from gymnasium.envs.registration import register
import gymnasium as gym
from q_learn_code.agent.Agent import Agent
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle as pk
import json
import time

In [2]:
city = 'Perth'

In [3]:
data = pd.read_csv("data_process/_RawData/" + city + "/POIs.csv")
quantity_of_POIS = len(data)

POI_to_ID = {}
ID_to_POI = {}
index = 0
for poi in list(data['poiID']):
    POI_to_ID[poi] = index
    ID_to_POI[index] = poi
    index = index + 1

In [4]:
register(
     id="MARCPTR",
     entry_point="q_learn_code.envs:MARCPTR",
     max_episode_steps=300,
)

In [5]:
env = gym.make('q_learn_code.envs:MARCPTR')

In [6]:
env.set_size(quantity_of_POIS)

25

In [7]:
file = open('data_process/' + city + '/sequence', 'rb')
sequence = pk.load(file)

startpoint = {}
for item in sequence:
    start = sequence[item][0]
    end = sequence[item][-1]
    startpoint[item] = (start, end)
    
#startpoint

In [8]:
len(startpoint)

159

In [9]:
number_of_users = len(startpoint)

In [10]:
def train_model(agent, n_episodes):
    #env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
    total_rw = []
    for episode in range(n_episodes):
        obs, info = env.reset()
        done = False

        # play one episode
        rw = 0
        while not done:
            action = agent.get_action(obs['agent'], env)

            next_obs, reward, terminated, truncated, info = env.step(action)
            
            rw = rw + reward

            # update the agent
            agent.update(obs['agent'], action, reward, terminated, next_obs['agent'])

            # update if the environment is done and the current obs
            done = terminated or truncated

            obs = next_obs
        total_rw.append(rw)
        #print(rw)
        #print(reward)

        agent.decay_epsilon()
    return np.array(total_rw)

In [11]:
# hyperparameters
learning_rate = 0.01
n_episodes = 100
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)  # reduce the exploration over time
final_epsilon = 0.1
#################


In [12]:
start_time = time.time()

reward = 0
Q_TABLES = {}
for item in tqdm(startpoint):
    agent = Agent(learning_rate=learning_rate,initial_epsilon=start_epsilon,
              epsilon_decay=epsilon_decay,final_epsilon=final_epsilon, q_table_size=quantity_of_POIS)
    
    env.reset(start=POI_to_ID[startpoint[item][0]], end=POI_to_ID[startpoint[item][1]])
    env.define_user(item, city)
    reward = reward + train_model(agent, n_episodes)
    Q_TABLES[item] = agent.q_values

final_time = time.time()    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 159/159 [00:23<00:00,  6.73it/s]


In [13]:
f = open("results/" + city + "/time", "a")
f.writelines("train_q_learning=" + str(final_time - start_time) + "\n")
f.close()

In [14]:
reward = reward / number_of_users

In [15]:
with open("results/" + city + "/reward","wb") as file:
    pk.dump(reward, file)

In [16]:
recomendation = {}

In [17]:
def get_shortest_path(q_values, start, end):
    if(start == end):
        return [start]
    next_obs, info = env.reset(start=start, end=end)
    #print(next_obs)
    done = False
    shortest_path = []
    shortest_path.append(start)
    while not done:
        action_index = np.argmax(q_values[next_obs['agent'][1]])
        #print('proximo POI', action_index)
        while(action_index in shortest_path):
            q_values[:, np.argmax(q_values[next_obs['agent'][1]])] = -1000
            action_index = np.argmax(q_values[next_obs['agent'][1]])
            #print('proximo POI - inside while', action_index)
        next_obs, reward, terminated, truncated, info = env.step(action_index)
        #print(next_obs, terminated)
        shortest_path.append(next_obs['agent'][1])
        done = terminated or truncated
    return shortest_path

In [18]:
start_time = time.time()

biggest_seq = 0
recomendation = {}
for item in startpoint:
    q_values = Q_TABLES[item].copy()
    env.reset(start=POI_to_ID[startpoint[item][0]], end=POI_to_ID[startpoint[item][1]])
    sp = get_shortest_path(q_values, POI_to_ID[startpoint[item][0]], POI_to_ID[startpoint[item][1]])
    if len(sp) > biggest_seq:
        biggest_seq = len(sp)
    sp = [ID_to_POI[item] for item in sp]
    recomendation[item] = (sequence[item], sp)

final_time = time.time() 

In [19]:
f = open("results/" + city + "/time", "a")
f.writelines("recommend=" + str(final_time - start_time) + "\n")
f.close()

In [20]:
recomendation

{'11146442@N00': ([1, 7, 6, 23], [1, 14, 3, 12, 20, 22, 11, 7, 17, 23]),
 '11587194@N06': ([18, 6, 1],
  [18, 14, 23, 4, 2, 13, 17, 10, 22, 25, 15, 11, 5, 9, 1]),
 '11845746@N00': ([13], [13]),
 '12199582@N00': ([12], [12]),
 '12288823@N07': ([20], [20]),
 '15213592@N03': ([1, 22, 7, 18, 19],
  [1,
   17,
   14,
   25,
   13,
   4,
   9,
   10,
   22,
   20,
   3,
   24,
   12,
   5,
   6,
   2,
   18,
   15,
   11,
   23,
   16,
   21,
   19]),
 '16105263@N00': ([15], [15]),
 '17295541@N02': ([20], [20]),
 '17611055@N00': ([3], [3]),
 '19229064@N00': ([22, 6], [22, 14, 13, 15, 11, 23, 12, 1, 17, 16, 2, 6]),
 '19604783@N00': ([6], [6]),
 '20385902@N03': ([1], [1]),
 '20692723@N07': ([6], [6]),
 '20724439@N00': ([19], [19]),
 '21156246@N00': ([22, 6],
  [22, 3, 21, 1, 4, 16, 12, 23, 10, 11, 7, 14, 13, 24, 9, 15, 17, 8, 5, 6]),
 '21717623@N04': ([9, 17, 12, 1],
  [9, 14, 13, 24, 4, 11, 5, 6, 3, 12, 15, 23, 1]),
 '22563879@N04': ([17], [17]),
 '22872308@N00': ([22], [22]),
 '22997058@N02'

In [21]:
user_app_demand = {}
file = open("data_process/_RawData/" + city + "/user_apps.in", "r")
data = file.readlines()
for i in data:
    user = i.rsplit('"')[0].rsplit()[0]
    app = i.rsplit('"')[1]
    
    if(app == 'Augmented reality'):
        user_app_demand[user] = {'bandwidth':(1, 10), 'processing':(0.1, 1)}
    elif(app=='Video streaming'):
        user_app_demand[user] = {'bandwidth':(1.5, 25), 'processing':(0, 0)}
    elif(app=='Video streaming and augmented reality'):
        user_app_demand[user] = {'bandwidth':(2.5, 35), 'processing':(0.1, 1)}
    elif(app=='No application'):
        user_app_demand[user] = {'bandwidth':(0,0), 'processing':(0, 0)}

In [22]:
data = pd.read_csv("data_process/_RawData/" + city + "/touristsVisits.csv")
users_id = data['userID'].unique()
for i in users_id:
    if i not in user_app_demand:
        user_app_demand[i] = {'bandwidth': (0, 0), 'processing': (0, 0)}

In [23]:
def allocate_resources(POI_BANDWIDTH_CAPACITY,  MEC_CAPACITY):
    band_demand_for_user = dict.fromkeys(list(startpoint.keys()), None)
    mec_demand_for_user = dict.fromkeys(list(startpoint.keys()), None)
    for item in list(startpoint.keys()):
        band_demand_for_user[item] = []
        mec_demand_for_user[item] = []

    for t in range(biggest_seq):
        for p in range(quantity_of_POIS):

            list_of_user = []
            list_of_b_dem = []
            list_of_m_dem = []
            for item in recomendation:
                #print(t)
                #print(recomendation[item][1][t])
                if (len(recomendation[item][1]) > t) and recomendation[item][1][t] == ID_to_POI[p]:
                    #print(recomendation[item][1][t], ID_to_POI[p])
                    list_of_user.append(item)
                    list_of_b_dem.append(user_app_demand[item]['bandwidth'][1])
                    list_of_m_dem.append(user_app_demand[item]['processing'][1])

            factor = 0.05
            while(np.sum(list_of_b_dem)) > POI_BANDWIDTH_CAPACITY:
                list_of_b_dem = [x * (1-factor) for x in list_of_b_dem]
                factor = factor + 0.05

            factor = 0.05
            while(np.sum(list_of_m_dem)) > MEC_CAPACITY:
                list_of_m_dem = [x * (1-factor) for x in list_of_m_dem]
                factor = factor + 0.05     

            for dem in range(len(list_of_b_dem)):
                band_demand_for_user[list_of_user[dem]].append(list_of_b_dem[dem])
                mec_demand_for_user[list_of_user[dem]].append(list_of_m_dem[dem])
    
    return (band_demand_for_user, mec_demand_for_user)


In [24]:
POI_BANDWIDTH_CAPACITY = [75, 150, 300]
MEC_CAPACITY = 37.5

In [25]:
start_time = time.time()

band_demand_for_user, mec_demand_for_user = allocate_resources(POI_BANDWIDTH_CAPACITY[0], MEC_CAPACITY)

FINAL_RESULTS = {}

for item in users_id:
    FINAL_RESULTS[item] = {'original':recomendation[item][0], 
                           'recommendation':recomendation[item][1],
                           'demand_bandwidth':user_app_demand[item]['bandwidth'],
                           'allocated_bandwidth':band_demand_for_user[item],
                           'demand_mec':user_app_demand[item]['processing'],
                           'allocated_mec':mec_demand_for_user[item]}
    
with open('results/' + city + '/' + str(POI_BANDWIDTH_CAPACITY[0]) + '_result.json', 'w') as fp:
    json.dump(FINAL_RESULTS, fp, indent=4, default=int)
    
final_time = time.time()

f = open("results/" + city + "/time", "a")
f.writelines("allocate_75=" + str(final_time - start_time) + "\n")
f.close()

In [26]:
start_time = time.time()

band_demand_for_user, mec_demand_for_user = allocate_resources(POI_BANDWIDTH_CAPACITY[1], MEC_CAPACITY)

FINAL_RESULTS = {}

for item in users_id:
    FINAL_RESULTS[item] = {'original':recomendation[item][0], 
                           'recommendation':recomendation[item][1],
                           'demand_bandwidth':user_app_demand[item]['bandwidth'],
                           'allocated_bandwidth':band_demand_for_user[item],
                           'demand_mec':user_app_demand[item]['processing'],
                           'allocated_mec':mec_demand_for_user[item]}
    
with open('results/' + city + '/' + str(POI_BANDWIDTH_CAPACITY[1]) + '_result.json', 'w') as fp:
    json.dump(FINAL_RESULTS, fp, indent=4, default=int)
    
final_time = time.time()

f = open("results/" + city + "/time", "a")
f.writelines("allocate_150=" + str(final_time - start_time) + "\n")
f.close()

In [27]:
start_time = time.time()

band_demand_for_user, mec_demand_for_user = allocate_resources(POI_BANDWIDTH_CAPACITY[2], MEC_CAPACITY)

FINAL_RESULTS = {}

for item in users_id:
    FINAL_RESULTS[item] = {'original':recomendation[item][0], 
                           'recommendation':recomendation[item][1],
                           'demand_bandwidth':user_app_demand[item]['bandwidth'],
                           'allocated_bandwidth':band_demand_for_user[item],
                           'demand_mec':user_app_demand[item]['processing'],
                           'allocated_mec':mec_demand_for_user[item]}
    
with open('results/' + city + '/' + str(POI_BANDWIDTH_CAPACITY[2]) + '_result.json', 'w') as fp:
    json.dump(FINAL_RESULTS, fp, indent=4, default=int)
    
final_time = time.time()

f = open("results/" + city + "/time", "a")
f.writelines("allocate_300=" + str(final_time - start_time) + "\n")
f.close()